IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\NO_FAULT6


In [3]:
guasto=0

VOLO m6 NO FAULT

In [4]:
rcou_m6_nofault = pd.read_csv("RCOU.csv")
rcou_m6_nofault = rcou_m6_nofault.astype("float64")
rcou_m6_nofault = rcou_m6_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_nofault = rcou_m6_nofault[((rcou_m6_nofault['C9'] >= potenza) & (rcou_m6_nofault['C10'] >= potenza) & (rcou_m6_nofault['C11'] >= potenza) & (rcou_m6_nofault['C12'] >= potenza) & (rcou_m6_nofault['C13'] >= potenza) & (rcou_m6_nofault['C14']>= potenza))]
rcou_m6_nofault=rcou_m6_nofault.reset_index(drop=True)
display(rcou_m6_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,73585436.0,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0
1,73609813.0,1415.0,1428.0,1402.0,1439.0,1470.0,1366.0
2,73620421.0,1415.0,1438.0,1423.0,1431.0,1483.0,1364.0
3,73663445.0,1448.0,1452.0,1468.0,1431.0,1530.0,1356.0
4,73673952.0,1451.0,1461.0,1462.0,1449.0,1523.0,1379.0
...,...,...,...,...,...,...,...
5481,141114087.0,1562.0,1440.0,1526.0,1481.0,1478.0,1529.0
5482,141125072.0,1574.0,1422.0,1525.0,1479.0,1490.0,1516.0
5483,141136462.0,1603.0,1378.0,1547.0,1451.0,1544.0,1454.0
5484,141345328.0,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0


In [5]:
min=rcou_m6_nofault['TimeUS'][0]
max=rcou_m6_nofault['TimeUS'][len(rcou_m6_nofault)-1]
print(max)
print(min)

141376898.0
73585436.0


In [6]:
xkf1_m6_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m6_nofault = xkf1_m6_nofault.astype("float64")
xkf1_m6_nofault = xkf1_m6_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_nofault = xkf1_m6_nofault[((xkf1_m6_nofault['TimeUS'] >= min) & (xkf1_m6_nofault['TimeUS'] <= max	))]
xkf1_m6_nofault = xkf1_m6_nofault.reset_index(drop=True)
print(xkf1_m6_nofault)

           TimeUS  Roll  Pitch    Yaw
0      73597012.0  1.77  -1.55  15.93
1      73607972.0  1.70  -1.53  15.96
2      73617984.0  1.62  -1.51  15.99
3      73628863.0  1.62  -1.52  16.00
4      73638809.0  1.63  -1.51  16.01
...           ...   ...    ...    ...
6139  141325367.0 -0.09  -2.57   4.16
6140  141343293.0 -0.16  -2.48   3.98
6141  141353420.0 -0.13  -2.25   3.91
6142  141364373.0 -0.15  -2.06   3.89
6143  141374850.0 -0.24  -1.93   3.86

[6144 rows x 4 columns]


In [7]:
att_m6_nofault = pd.read_csv("ATT.csv")
att_m6_nofault = att_m6_nofault.astype("float64")
att_m6_nofault = att_m6_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_nofault = att_m6_nofault[((att_m6_nofault['TimeUS'] >= min) & (att_m6_nofault['TimeUS'] <= max	))]
att_m6_nofault=att_m6_nofault.reset_index(drop=True)
print(att_m6_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303.0     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051.0     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815.0     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059.0     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504.0     0.01  1.63     -0.53  -1.51   18.84  16.01
...           ...      ...   ...       ...    ...     ...    ...
6139  141327089.0    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718.0    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891.0    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784.0    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080.0     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m6_nofault)>len(xkf1_m6_nofault):
    to_add=att_m6_nofault[len(xkf1_m6_nofault):]
    att_m6_nofault=att_m6_nofault[:len(xkf1_m6_nofault)]

if len(xkf1_m6_nofault)>len(att_m6_nofault):
    to_add=xkf1_m6_nofault[len(att_m6_nofault):]

for idx,i in enumerate(att_m6_nofault['Roll']):
    if(xkf1_m6_nofault['Roll'][idx] != i):
        att_m6_nofault['Roll'][idx] = ((att_m6_nofault['Roll'][idx] + xkf1_m6_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Pitch']):
    if(xkf1_m6_nofault['Pitch'][idx] != i):
        att_m6_nofault['Pitch'][idx] = ((att_m6_nofault['Pitch'][idx] + xkf1_m6_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Yaw']):
    if(xkf1_m6_nofault['Yaw'][idx] != i):
        att_m6_nofault['Yaw'][idx] = ((att_m6_nofault['Yaw'][idx] + xkf1_m6_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_nofault=pd.concat([att_m6_nofault,to_add])

print(att_m6_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303.0     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051.0     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815.0     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059.0     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504.0     0.01  1.63     -0.53  -1.51   18.84  16.01
...           ...      ...   ...       ...    ...     ...    ...
6139  141327089.0    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718.0    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891.0    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784.0    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080.0     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m6_nofault = pd.read_csv("ESC_0.csv")
esc_0_m6_nofault = esc_0_m6_nofault.astype("float64")
esc_0_m6_nofault = esc_0_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m6_nofault = esc_0_m6_nofault[((esc_0_m6_nofault['TimeUS'] >= min) & (esc_0_m6_nofault['TimeUS'] <= max))]
esc_0_m6_nofault=esc_0_m6_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m6_nofault = pd.read_csv("ESC_1.csv")
esc_1_m6_nofault = esc_1_m6_nofault.astype("float64")
esc_1_m6_nofault = esc_1_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m6_nofault = esc_1_m6_nofault[((esc_1_m6_nofault['TimeUS'] >= min) & (esc_1_m6_nofault['TimeUS'] <= max))]
esc_1_m6_nofault=esc_1_m6_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m6_nofault = pd.read_csv("ESC_2.csv")
esc_2_m6_nofault = esc_2_m6_nofault.astype("float64")
esc_2_m6_nofault = esc_2_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m6_nofault = esc_2_m6_nofault[((esc_2_m6_nofault['TimeUS'] >= min) & (esc_2_m6_nofault['TimeUS'] <= max))]
esc_2_m6_nofault=esc_2_m6_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m6_nofault = pd.read_csv("ESC_3.csv")
esc_3_m6_nofault = esc_3_m6_nofault.astype("float64")
esc_3_m6_nofault = esc_3_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m6_nofault = esc_3_m6_nofault[((esc_3_m6_nofault['TimeUS'] >= min) & (esc_3_m6_nofault['TimeUS'] <= max))]
esc_3_m6_nofault=esc_3_m6_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m6_nofault = pd.read_csv("ESC_4.csv")
esc_4_m6_nofault = esc_4_m6_nofault.astype("float64")
esc_4_m6_nofault = esc_4_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m6_nofault = esc_4_m6_nofault[((esc_4_m6_nofault['TimeUS'] >= min) & (esc_4_m6_nofault['TimeUS'] <= max))]
esc_4_m6_nofault=esc_4_m6_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m6_nofault = pd.read_csv("ESC_5.csv")
esc_5_m6_nofault = esc_5_m6_nofault.astype("float64")
esc_5_m6_nofault = esc_5_m6_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m6_nofault = esc_5_m6_nofault[((esc_5_m6_nofault['TimeUS'] >= min) & (esc_5_m6_nofault['TimeUS'] <= max))]
esc_5_m6_nofault=esc_5_m6_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_nofault = pd.read_csv("IMU_0.csv")
imu_0_m6_nofault = imu_0_m6_nofault.astype("float64")
imu_0_m6_nofault = imu_0_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_nofault = imu_0_m6_nofault[((imu_0_m6_nofault['TimeUS'] >= min) & (imu_0_m6_nofault['TimeUS'] <= max))]

imu_1_m6_nofault = pd.read_csv("IMU_1.csv")
imu_1_m6_nofault = imu_1_m6_nofault.astype("float64")
imu_1_m6_nofault = imu_1_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_nofault = imu_1_m6_nofault[((imu_1_m6_nofault['TimeUS'] >= min) & (imu_1_m6_nofault['TimeUS'] <= max))]

imu_2_m6_nofault = pd.read_csv("IMU_2.csv")
imu_2_m6_nofault = imu_2_m6_nofault.astype("float64")
imu_2_m6_nofault = imu_2_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_nofault = imu_2_m6_nofault[((imu_2_m6_nofault['TimeUS'] >= min) & (imu_2_m6_nofault['TimeUS'] <= max))]

imu_m6_nofault = pd.concat((imu_0_m6_nofault, imu_1_m6_nofault, imu_2_m6_nofault))
imu_m6_nofault=imu_m6_nofault.groupby(imu_m6_nofault.TimeUS, as_index=False).mean()

display(imu_m6_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,73587431.0,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330
1,73590478.0,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719
2,73592092.0,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333
3,73598199.0,0.136942,-0.077143,-0.004878,-0.089322,0.035817,-9.557465
4,73604174.0,-0.115818,0.011950,0.043407,-0.420610,-0.265488,-9.875872
...,...,...,...,...,...,...,...
23374,141365515.0,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230
23375,141368756.0,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680
23376,141370810.0,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503
23377,141373732.0,-0.240294,0.277462,-0.019804,-1.017304,-0.481268,-10.041165


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m6_nofault = pd.merge_ordered(imu_m6_nofault,att_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m6_nofault = pd.merge_ordered(m6_nofault, esc_0_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_1_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_2_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_3_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_4_m6_nofault)
m6_nofault = pd.merge_ordered(m6_nofault, esc_5_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")
m6_nofault = pd.merge_ordered(m6_nofault, rcou_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m6_nofault["TimeUS"] = m6_nofault["TimeUS"] - m6_nofault.iloc[0]["TimeUS"]
m6_nofault["TimeUS"] = m6_nofault["TimeUS"].astype(int)
m6_nofault["Guasto"] = guasto

In [13]:
last_cell = m6_nofault.iloc[-1, m6_nofault.columns.get_loc('TimeUS')]
print(last_cell)
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])


m6_nofault_final = pd.merge_ordered(m6_nofault,df)
m6_nofault_final=m6_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m6_nofault_final)


m6_nofault_final = m6_nofault_final[m6_nofault_final.TimeUS.isin(to_be)]

m6_nofault_final

67791462
         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.147274 -0.138449 -0.001893 -0.119606  0.091852  -9.635330   
1          1995  0.147274 -0.138449 -0.001893 -0.119606  0.091852  -9.635330   
2          2313  0.147274 -0.138449 -0.001893 -0.119606  0.091852  -9.635330   
3          2857  0.147274 -0.138449 -0.001893 -0.119606  0.091852  -9.635330   
4          5042  0.092451 -0.148777  0.007550 -0.255108 -0.065997  -9.740719   
...         ...       ...       ...       ...       ...       ...        ...   
62169  67788296 -0.240294  0.277462 -0.019804 -1.017304 -0.481268 -10.041165   
62170  67790524 -0.154242  0.247645 -0.015770 -1.007628 -0.310841 -10.326050   
62171  67790644 -0.154242  0.247645 -0.015770 -1.007628 -0.310841 -10.326050   
62172  67790896 -0.154242  0.247645 -0.015770 -1.007628 -0.310841 -10.326050   
62173  67791462 -0.154242  0.247645 -0.015770 -1.007628 -0.310841 -10.326050   

       DesRoll  Roll  DesPitch

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
3,2857,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
5,5714,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
7,8571,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
8,11428,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62160,67779468,-0.170500,0.261187,-0.083874,-1.089565,-1.073007,-9.274010,-0.25,-0.13,-2.76,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
62164,67782325,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
62166,67785182,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
62168,67788039,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0


In [14]:
from datetime import timedelta
m6_nofault_final=m6_nofault_final.reset_index(drop=True)
m6_nofault_final['TimeUS'] = pd.to_datetime(m6_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
1,00:00:00.002857,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
2,00:00:00.005714,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
3,00:00:00.008571,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
4,00:00:00.011428,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,...,2.98,3581.0,2.34,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23724,00:01:07.779468,-0.170500,0.261187,-0.083874,-1.089565,-1.073007,-9.274010,-0.25,-0.13,-2.76,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23725,00:01:07.782325,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23726,00:01:07.785182,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23727,00:01:07.788039,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503,-0.05,-0.15,-2.88,...,0.43,5148.0,0.47,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_nofault_final)/350)):
        V1=m6_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29784\2215761558.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29784\2215761558.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.061274,0.005950,0.034892,0.098425,0.0,88.0,12.505537,2.495923,0.035329,...,13196.353007,3.430969,0.581929,-1.397343,3.514515,109.0,108.0,39.862043,16.996902,0
1,1.0,-0.061302,0.005933,0.042842,0.098357,0.0,99.0,12.453780,2.371632,0.035380,...,13221.535787,3.433381,0.582319,-1.391687,3.516926,109.0,108.0,39.582214,16.419804,0
2,2.0,-0.061041,0.005940,0.036872,0.098231,0.0,94.0,12.502659,2.343510,0.035068,...,13162.389441,3.435779,0.582695,-1.386039,3.519321,109.0,108.0,39.290369,15.797818,0
3,3.0,-0.060840,0.005938,0.036149,0.098094,0.0,91.0,12.530302,2.289410,0.034748,...,13023.189192,3.438164,0.583058,-1.380398,3.521702,109.0,108.0,38.986744,15.132373,0
4,4.0,-0.061014,0.005932,0.031922,0.098172,0.0,96.0,12.424397,2.097929,0.034754,...,12808.845793,3.440535,0.583407,-1.374764,3.524067,109.0,108.0,38.671575,14.425026,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62.0,-0.048472,0.006254,-0.082119,0.092673,0.0,119.0,13.705592,4.427470,0.036188,...,13757.613392,3.538935,0.562466,-1.024534,3.617342,108.0,109.0,20.020166,14.751039,0
63,63.0,-0.048289,0.006253,-0.081993,0.092571,0.0,118.0,13.703035,4.344276,0.036631,...,13259.009540,3.540628,0.562292,-1.018466,3.618975,110.0,111.0,20.217965,15.384214,0
64,64.0,-0.048212,0.006240,-0.075446,0.092463,0.0,119.0,13.655833,4.213446,0.037145,...,12751.414008,3.542313,0.562112,-1.012392,3.620599,108.0,109.0,20.375260,16.025541,0
65,65.0,-0.048179,0.006225,-0.068067,0.092367,0.0,130.0,13.590221,4.061607,0.037632,...,12235.461935,3.543990,0.561928,-1.006311,3.622215,108.0,109.0,20.489324,16.674333,0


In [18]:
path_file = path_file.replace(r"M6\NO_FAULT6", "")
os.chdir(path_file)
df_merged.to_csv('m6_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
